In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report


from sklearn.linear_model import LinearRegression
# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import os
os.listdir('/kaggle/input/salary-data')
df=pd.read_csv('/kaggle/input/salary-data/Salary_Data.csv')
df.head()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/salary-data/Salary_Data.csv


,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


In [2]:
##categorical to numerical
from sklearn.preprocessing import LabelEncoder
genderLE=LabelEncoder()
df['Gender']=genderLE.fit_transform(df['Gender'])

educationLE=LabelEncoder()
df['Education Level']=educationLE.fit_transform(df['Education Level'])

jobLE=LabelEncoder()
df['Job Title']=jobLE.fit_transform(df['Job Title'])


In [3]:
df=df.dropna(axis=0)
#print(df)
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,1,0,177,5.0,90000.0
1,28.0,0,3,18,3.0,65000.0
2,45.0,1,5,145,15.0,150000.0
3,36.0,0,0,116,7.0,60000.0
4,52.0,1,3,26,20.0,200000.0


In [4]:
X=df.drop(columns='Salary')
y=df['Salary']

In [5]:
x_train,x_test,y_train,y_test=train_test_split(X,y,random_state=21,test_size=0.2)
#print("x train shape and y train shape respectively: ",x_train.shape,y_train.shape)


In [6]:
from sklearn.tree import DecisionTreeRegressor
tree=DecisionTreeRegressor()

In [7]:
parameters={'max_depth': [3, 5, 7],'min_samples_split': [5, 10, 15]}

In [8]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(estimator=tree,param_grid=parameters,cv=5,scoring='r2',)
grid.fit(x_train,y_train)


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [3, 5, 7],
                         'min_samples_split': [5, 10, 15]},
             scoring='r2')

In [9]:
print(grid.best_estimator_)
print(grid.best_score_)
print(grid.score(x_test,y_test))  

DecisionTreeRegressor(max_depth=7, min_samples_split=5)
0.9197837221573962
0.9074043062174926


In [10]:
##Linear Regression Using Randomized Search
from sklearn.model_selection import GridSearchCV
lr=LinearRegression()
dist={'fit_intercept':[True,False]}
rs=GridSearchCV(estimator=lr,param_grid=dist,cv=3,scoring='r2')

In [11]:
rs.fit(x_train,y_train)
print(rs.best_estimator_)
print(rs.best_score_)
##low score. not suitable

LinearRegression()
0.6676393958338905


In [12]:
##Random Forest Regressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
rf=RandomForestRegressor()
params={'n_estimators':randint(5,100),'max_depth':randint(3,50),'min_samples_leaf':randint(2,10)}
rs2=RandomizedSearchCV(estimator=rf,param_distributions=params,n_iter=20,scoring='r2',random_state=19,cv=3)


In [13]:
rs2.fit(x_train,y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=20,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f4304665d20>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f4304666ad0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f43049784c0>},
                   random_state=19, scoring='r2')

In [14]:
print(rs2.best_estimator_)
print(rs2.best_score_)

RandomForestRegressor(max_depth=42, min_samples_leaf=2, n_estimators=44)
0.9673801857775127


In [15]:
y_pred=rs2.best_estimator_.predict(x_test)

In [16]:
from sklearn.metrics import mean_squared_error, r2_score
print(r2_score(y_test,y_pred))

0.9763728505081645
